# RAG (Retrieval-Augmented Generation)

* RAG will help me to overcome the above problem which is limited context and static knowledge-doesn't automatically update when new events, aftcs occur.

* How it will help?

    - As I upload a document it will fetch a relevant information from that document and provide me result related to the query and the context. In simple terms, it allos to use external knowledge that it didn't learn while training.

* To Create a RAG-based agent,I first need to create knowledge base, which is a repository of all the documents or structured data.

### RAG based application is comprise of four main components:

    1. Document Loader
    2. Text splitter
    3. Vector Databases
    4. Retrievers


## Document Loader 
First thing to start with is, How do we load a document. A document can be pdf, word file, or any file.

* Data can be coming from various sources, such as pdf, text, google drive, slack, amazon s3, and the chances are high that each source might have different format. LangChain providea common interface for reading data regardless of the sources
* There are hundreds of document loaders. 

## TextLoader

* It is Commonly used Document loader and very simple in LangChain, converting them into LangChain Document Objects by reading plain text files.
* It only works with .txt files
* Ideal for loading logs, scraped texts, transcripts, code snippets, or any plain text data.

In [ ]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader("exaple.txt", encoding="utf-8")

documents = loader.load()
print(type(documents))
print(len(documents))
print(documents[0])


<class 'list'>
1
page_content='A widespread research field in regenerative medicine is electrical stimulation (ES),
and its impact on tissue and cells, such as bone [1]. In 1957 the piezoelectric properties of
bone were described [2]. As the bone healing process takes place under the mechanical
strain of the bone [3], an electric field is generated in the bone in vitro and in vivo [4].
Externally applied electric fields were shown to contribute to bone deposition and osteoblast
differentiation and proliferation [5]. The living cell has a membrane potential that indicates
the electrical potential difference between the intracellular and extracellular space, which
is generated by the transport of ions through ion channels and ion transporters [6]. The
membrane potential is always negative because of the higher negative ion concentration
inside the cell [7,8]. Primary osteoblasts revealed a membrane potential of −60 mV [9], MG-
63 osteoblasts demonstrated a negatively charged cell surface

In [5]:
print(documents[0].page_content)

A widespread research field in regenerative medicine is electrical stimulation (ES),
and its impact on tissue and cells, such as bone [1]. In 1957 the piezoelectric properties of
bone were described [2]. As the bone healing process takes place under the mechanical
strain of the bone [3], an electric field is generated in the bone in vitro and in vivo [4].
Externally applied electric fields were shown to contribute to bone deposition and osteoblast
differentiation and proliferation [5]. The living cell has a membrane potential that indicates
the electrical potential difference between the intracellular and extracellular space, which
is generated by the transport of ions through ion channels and ion transporters [6]. The
membrane potential is always negative because of the higher negative ion concentration
inside the cell [7,8]. Primary osteoblasts revealed a membrane potential of −60 mV [9], MG-
63 osteoblasts demonstrated a negatively charged cell surface of −15.6 mV [10]. Due to the
a

In [7]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

load_dotenv()

chat_model = init_chat_model("openai:gpt-4o-mini")

prompt = PromptTemplate(template = "Summarise the following text: \n\n{text}", input_variables=["text"])

parser = StrOutputParser()

chain = prompt | chat_model | parser

print(chain.invoke({"text": documents[0].page_content}))

Electrical stimulation (ES) is a prominent area of research in regenerative medicine, particularly regarding its effects on bone tissue and cells. Since the discovery of bone's piezoelectric properties in 1957, it has been recognized that mechanical strain in bone generates electric fields that influence healing processes, such as bone deposition and the differentiation of osteoblasts. Living cells have a negative membrane potential due to ion transport, which can be influenced by external electric fields. In response to bone injuries, a wound potential is created to recruit precursor cells for healing.

Research suggests that ES can affect various cellular processes, including cell adhesion, proliferation, and differentiation. Common methods for applying ES involve direct contact stimulation, where electrodes transmit current to cultured cells. This stimulation can alter cell membrane potential and trigger intracellular pathways, influencing calcium ion dynamics and reactive oxygen sp

## PyPDFLoader

* It is used to loadpdf documents and converts into LangChain Document Object.

In [11]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("cells-11-02650-v2.pdf")

docs = loader.load()

print(type(docs))
print(len(docs))
print(docs[0])


<class 'list'>
22
page_content='Citation: Staehlke, S.; Bielfeldt, M.;
Zimmermann, J.; Gruening, M.;
Barke, I.; Freitag, T.; Speller, S.; Van
Rienen, U.; Nebe, B. Pulsed Electrical
Stimulation Affects Osteoblast
Adhesion and Calcium Ion Signaling.
Cells 2022, 11, 2650. https://doi.org/
10.3390/cells11172650
Academic Editor: Ohad Medalia
Received: 28 July 2022
Accepted: 23 August 2022
Published: 25 August 2022
Publisher’s Note:MDPI stays neutral
with regard to jurisdictional claims in
published maps and institutional afﬁl-
iations.
Copyright: © 2022 by the authors.
Licensee MDPI, Basel, Switzerland.
This article is an open access article
distributed under the terms and
conditions of the Creative Commons
Attribution (CC BY) license (https://
creativecommons.org/licenses/by/
4.0/).
cells
Article
Pulsed Electrical Stimulation Affects Osteoblast Adhesion and
Calcium Ion Signaling
Susanne Staehlke 1, *
 , Meike Bielfeldt 1
 , Julius Zimmermann 2
 , Martina Gruening 1, Ingo Barke 3,
Thomas Fr

In [15]:
print(docs[13].page_content)

Cells 2022, 11, 2650 14 of 22
Figure 10. Inﬂuence of AC electrical stimulation (ES, 10 min) on MG-63 cell morphology and
spreading at 0.5, 4, and 24 h. ( a) Scanning electron microscopy of cells. Note that ES does not
inﬂuence the overall cell shape. Cells start to spread at 0.5 h, which is typical for MG-63s. (FE-SEM
Merlin compact, Carl Zeiss, 500×, scale bars = 10 µm). (b) Cell area analysis. Note that the different
AC-stimulation settings have no signiﬁcant impact on the time-dependent spreading compared to
unstimulated control cells (Control). (ImageJ, n = 40 cells, median ±min/max; one-way ANOVA
posthoc uncorrected Fisher’s LSD, *p < 0.05).
Table 2. Cell area values of 10 min AC-stimulated MG-63 osteoblastic cells (mean ±s.e.m.).
Time after AC
Stimulation
Control
(in µm2)
1 V_20 Hz
(in µm2)
1 V_7.9 Hz
(in µm2)
5 V_20 Hz
(in µm2)
5 V_7.9 Hz
(in µm2)
0.5 h 319 ±5.1 295 ±4.9 311.3 ±10.0 322.6 ±7.1 375.4 ±5.7
4 h 903 ±16.4 1075.9 ±16.3 947.7 ±10.9 890.6 ±12.2 955.7 ±13.1
24 h 1422.7 

So, basically it has ignored all the figures from the pdf and tabular data also not structured properly.

In [14]:
print(docs[13].metadata)

{'producer': 'pdfTeX-1.40.21', 'creator': 'LaTeX with hyperref', 'creationdate': '2022-08-29T13:52:47+08:00', 'author': 'Susanne Staehlke, Meike Bielfeldt, Julius Zimmermann, Martina Gruening, Ingo Barke, Thomas Freitag, Sylvia Speller, Ursula Van Rienen and Barbara Nebe', 'keywords': 'electrical stimulation; electric field strength; field simulation; AC-stimulated liquid; osteoblasts adhesion; spreading; calcium ions; reactive oxygen species; confocal microscopy; scanning electron microscopy', 'moddate': '2022-08-29T08:02:59+02:00', 'subject': 'An extensive research field in regenerative medicine is electrical stimulation (ES) and its impact on tissue and cells. The mechanism of action of ES, particularly the role of electrical parameters like intensity, frequency, and duration of the electric field, is not yet fully understood. Human MG-63 osteoblasts were electrically stimulated for 10 min with a commercially available multi-channel system (IonOptix). We generated alternating curren

### Limitations

* Not good with complex layouts

### PyMuPDFLoader

In [17]:
from langchain_community.document_loaders import PyMuPDFLoader
loader = PyMuPDFLoader("cells-11-02650-v2.pdf") 
docs = loader.load()
print(type(docs))
print(len(docs))


<class 'list'>
22


In [18]:
print(docs[13].page_content)

Cells 2022, 11, 2650
14 of 22
Figure 10. Inﬂuence of AC electrical stimulation (ES, 10 min) on MG-63 cell morphology and
spreading at 0.5, 4, and 24 h. (a) Scanning electron microscopy of cells. Note that ES does not
inﬂuence the overall cell shape. Cells start to spread at 0.5 h, which is typical for MG-63s. (FE-SEM
Merlin compact, Carl Zeiss, 500×, scale bars = 10 µm). (b) Cell area analysis. Note that the different
AC-stimulation settings have no signiﬁcant impact on the time-dependent spreading compared to
unstimulated control cells (Control). (ImageJ, n = 40 cells, median ± min/max; one-way ANOVA
posthoc uncorrected Fisher’s LSD, * p < 0.05).
Table 2. Cell area values of 10 min AC-stimulated MG-63 osteoblastic cells (mean ± s.e.m.).
Time after AC
Stimulation
Control
(in µm2)
1 V_20 Hz
(in µm2)
1 V_7.9 Hz
(in µm2)
5 V_20 Hz
(in µm2)
5 V_7.9 Hz
(in µm2)
0.5 h
319 ± 5.1
295 ± 4.9
311.3 ± 10.0
322.6 ± 7.1
375.4 ± 5.7
4 h
903 ± 16.4
1075.9 ± 16.3
947.7 ± 10.9
890.6 ± 12.2
955.7 ± 13.1


### Nope, This pdf loader is also not working well.

In [24]:
from langchain_unstructured import UnstructuredLoader
from unstructured.cleaners.core import clean_extra_whitespace
loader = UnstructuredLoader("cells-11-02650-v2.pdf",
post_processors=[clean_extra_whitespace],) 
docs = loader.load()
docs[0]

Document(metadata={'source': 'cells-11-02650-v2.pdf', 'coordinates': {'points': ((35.716, 50.23324000000002), (35.716, 84.25), (120.7579, 84.25), (120.7579, 50.23324000000002)), 'system': 'PixelSpace', 'layout_width': 595.276, 'layout_height': 841.89}, 'filename': 'cells-11-02650-v2.pdf', 'last_modified': '2025-11-26T15:19:55', 'page_number': 1, 'languages': ['eng'], 'filetype': 'application/pdf', 'category': 'Title', 'element_id': 'a8c76de821c337f4eb981ceede15be04'}, page_content='cells')

In [30]:
from langchain_community.document_loaders import UnstructuredPDFLoader
loader = UnstructuredPDFLoader("cells-11-02650-v2.pdf", mode = "elements")

docs = loader.load()
print(type(docs))
print(len(docs))


<class 'list'>
332


In [48]:
print(docs[0].metadata)

{'source': 'cells-11-02650-v2.pdf', 'coordinates': {'points': ((35.716, 50.23324000000002), (35.716, 84.25), (120.7579, 84.25), (120.7579, 50.23324000000002)), 'system': 'PixelSpace', 'layout_width': 595.276, 'layout_height': 841.89}, 'filename': 'cells-11-02650-v2.pdf', 'last_modified': '2025-11-26T15:19:55', 'page_number': 1, 'languages': ['eng'], 'filetype': 'application/pdf', 'category': 'Title', 'element_id': 'a8c76de821c337f4eb981ceede15be04'}


#### This line contains all the details of the pdf, where catagory is title, narrative text and so, below is printed categories and their contents

In [39]:
for i in range(len(docs)):
    print(i, docs[i].metadata.get("category"), docs[i].page_content)


0 Title cells
1 Title Article Pulsed Electrical Stimulation Affects Osteoblast Adhesion and Calcium Ion Signaling
2 UncategorizedText Susanne Staehlke 1,* Thomas Freitag 1, Sylvia Speller 3,4, Ursula Van Rienen 2,4,5
3 Title , Meike Bielfeldt 1
4 Title , Julius Zimmermann 2
5 UncategorizedText , Martina Gruening 1, Ingo Barke 3,
6 Title and Barbara Nebe 1,4
7 UncategorizedText 1 Department of Cell Biology, Rostock University Medical Center, 18057 Rostock, Germany 2 Department of Computer Science and Electrical Engineering, Institute of General Electrical Engineering,
8 UncategorizedText University of Rostock, 18059 Rostock, Germany Physics of Surfaces and Interfaces, Institute of Physics, University of Rostock, 18059 Rostock, Germany 4 Department Life, Light & Matter, Interdisciplinary Faculty, University of Rostock, 18059 Rostock, Germany 5 Department Aging of Individuals and Society, Interdisciplinary Faculty, University of Rostock,
9 UncategorizedText 3
10 Title 18059 Rostock, Germa

### The contents also has noise or unrequired data, so the page_content length is greater then certain length thenon;y it will show

In [43]:
long_elements = [doc for doc in docs if len(doc.page_content) > 100]

print(len(long_elements))
print(long_elements[4].page_content[:])


133
A widespread research ﬁeld in regenerative medicine is electrical stimulation (ES), and its impact on tissue and cells, such as bone [1]. In 1957 the piezoelectric properties of bone were described [2]. As the bone healing process takes place under the mechanical strain of the bone [3], an electric ﬁeld is generated in the bone in vitro and in vivo [4]. Externally applied electric ﬁelds were shown to contribute to bone deposition and osteoblast differentiation and proliferation [5]. The living cell has a membrane potential that indicates the electrical potential difference between the intracellular and extracellular space, which is generated by the transport of ions through ion channels and ion transporters [6]. The membrane potential is always negative because of the higher negative ion concentration inside the cell [7,8]. Primary osteoblasts revealed a membrane potential of −60 mV [9], MG- 63 osteoblasts demonstrated a negatively charged cell surface of −15.6 mV [10]. Due to the 

In [44]:
clean_docs = [d for d in docs if len(d.page_content.strip()) > 20]


In [45]:
clean_docs

[Document(metadata={'source': 'cells-11-02650-v2.pdf', 'coordinates': {'points': ((35.358, 105.87963859999991), (35.358, 155.67012479999983), (548.3250880000003, 155.67012479999983), (548.3250880000003, 105.87963859999991)), 'system': 'PixelSpace', 'layout_width': 595.276, 'layout_height': 841.89}, 'filename': 'cells-11-02650-v2.pdf', 'last_modified': '2025-11-26T15:19:55', 'page_number': 1, 'languages': ['eng'], 'filetype': 'application/pdf', 'category': 'Title', 'element_id': 'c9cfc5280c368b3f9df389da4a8fad3d'}, page_content='Article Pulsed Electrical Stimulation Affects Osteoblast Adhesion and Calcium Ion Signaling'),
 Document(metadata={'source': 'cells-11-02650-v2.pdf', 'coordinates': {'points': ((35.38799999999998, 168.33944559999986), (35.38799999999998, 192.71605160000001), (305.4715308, 192.71605160000001), (305.4715308, 168.33944559999986)), 'system': 'PixelSpace', 'layout_width': 595.276, 'layout_height': 841.89}, 'filename': 'cells-11-02650-v2.pdf', 'last_modified': '2025-1

### Here, All the Document object works as a chunk and it converted into ebeddings and stored into vector database. This embedding contains the context of the categories, ttables and image captions in the paper. so when user asks about specific thing from the table or image it then know where to retrieve answer.

1. Document objects as chunks

    * Each element extracted by UnstructuredPDFLoader (Title, NarrativeText, Table cell, Figure caption, UncategorizedText) becomes a chunk.

    * Very small elements may be combined into a larger chunk, depending on your chunking strategy.

2. Chunk → Embedding

    * Each chunk’s text content is converted into a numerical vector embedding.

    * The embedding encodes the semantic meaning of the text, so the model “understands” what the text is about.

    * Metadata, like category, page number, coordinates, table/figure info, is stored alongside the embedding, not inside it.

3. Stored in vector database

    * Embeddings + metadata are stored in a vector DB (like Chroma, FAISS, or Qdrant).

    * The metadata allows filtering or adding context, e.g., retrieving only Table data or Figure captions.

4. Query & Retrieval

    * When a user asks a question about a specific table or figure, RAG:

    * Converts the question to an embedding

    * Finds the most semantically similar chunks in the vector DB

    * Retrieves the text + metadata

    * Sends it to the LLM to generate the answer

5. Result

    * Even if the table data was originally UncategorizedText or the figure caption was a Title, the embedding captures the context and meaning, allowing RAG to locate the relevant chunk.

    * The LLM then “knows” where the answer comes from in the PDF.

### After a lot of resaearch, I understood now, why do we use different PDFLoaders.

* PyPDFLoader only extracts simple text where there is no structure necessary. It is fast, works for all pdfs but it doesn't help with tables, images, and columns.